In [1]:
import torch
import torch_geometric

train_data = torch.load('train_data.pt')
val_data = torch.load('val_data.pt')
test_data = torch.load('test_data.pt')

In [2]:
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score
from scipy.sparse.csgraph import shortest_path

import torch.nn.functional as F
from torch.nn import Conv1d, MaxPool1d, Linear, Dropout, BCEWithLogitsLoss, GRU

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, aggr, global_sort_pool
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix

In [3]:
from torch_geometric.data import Data
import numpy as np
def seal_processing(dataset, edge_label_index, y):
    data_list = []
    for src, dst in edge_label_index.t().tolist():
        sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph([src, dst], 2, dataset.edge_index, relabel_nodes=True)
        src, dst = mapping.tolist()

        # Remove target link from the subgraph
        mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
        mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
        sub_edge_index = sub_edge_index[:, mask1 & mask2]

        # Double-radius node labeling (DRNL)
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(sub_edge_index, num_nodes=sub_nodes.size(0)).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        # Calculate the distance between every node and the source target node
        d_src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
        d_src = np.insert(d_src, dst, 0, axis=0)
        d_src = torch.from_numpy(d_src)

        # Calculate the distance between every node and the destination target node
        d_dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
        d_dst = np.insert(d_dst, src, 0, axis=0)
        d_dst = torch.from_numpy(d_dst)

        # Calculate the label z for each node
        dist = d_src + d_dst
        z = 1 + torch.min(d_src, d_dst) + dist // 2 * (dist // 2 + dist % 2 - 1)
        z[src], z[dst], z[torch.isnan(z)] = 1., 1., 0.
        z = z.to(torch.long)

        # Concatenate node features and one-hot encoded node labels (with a fixed number of classes)
        node_labels = F.one_hot(z, num_classes=200).to(torch.float)
        node_emb = dataset.x[sub_nodes]
        node_x = torch.cat([node_emb, node_labels], dim=1)

        # Create data object
        data = Data(x=node_x, z=z, edge_index=sub_edge_index, y=y)
        data_list.append(data)

    return data_list

In [4]:
# Enclosing subgraphs extraction
train_pos_data_list = seal_processing(train_data, train_data.pos_edge_label_index, 1)
train_neg_data_list = seal_processing(train_data, train_data.neg_edge_label_index, 0)

In [5]:
#索引边界，和seal本身有关
#valid_mask = (val_data.neg_edge_label_index[0]) & (val_data.neg_edge_label_index[1])
#val_data.neg_edge_label_index = val_data.neg_edge_label_index[:, valid_mask]
val_pos_data_list = seal_processing(val_data, val_data.pos_edge_label_index, 1)
val_neg_data_list = seal_processing(val_data, val_data.neg_edge_label_index, 0)

In [6]:
#test_mask = (test_data.neg_edge_label_index[0]) & (test_data.neg_edge_label_index[1])
#test_data.neg_edge_label_index = test_data.neg_edge_label_index[:, test_mask]
test_pos_data_list = seal_processing(test_data, test_data.pos_edge_label_index, 1)
test_neg_data_list = seal_processing(test_data, test_data.neg_edge_label_index, 0)

In [7]:
train_dataset = train_pos_data_list + train_neg_data_list
val_dataset = val_pos_data_list + val_neg_data_list
test_dataset = test_pos_data_list + test_neg_data_list

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [9]:
class DGCNN(torch.nn.Module):
    def __init__(self, dim_in, k=50):
        super().__init__()

        # GCN layers
        self.gcn1 = GCNConv(dim_in, 32)
        self.gcn2 = GCNConv(32, 32)
        self.gcn3 = GCNConv(32, 32)
        self.gcn4 = GCNConv(32, 1)

        # Global sort pooling
        self.global_pool = aggr.SortAggregation(k=k)

        # Convolutional layers
        self.conv1 = Conv1d(1, 16, 97, 97)
        self.conv2 = Conv1d(16, 32, 5, 1)
        self.maxpool = MaxPool1d(2, 2)

        # Dense layers
        self.linear1 = Linear(672, 128)
        self.dropout = Dropout(0.5)
        self.linear2 = Linear(128, 1)

    def forward(self, x, edge_index, batch):
        # 1. Graph Convolutional Layers
        h1 = self.gcn1(x, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)

        # 2. Global sort pooling
        h = self.global_pool(h, batch)

        # 3. Traditional convolutional and dense layers
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()

        return h

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(train_dataset[0].num_features).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def train():
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)

@torch.no_grad()
def test(loader):
    model.eval()
    y_pred, y_true = [], []

    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        y_pred.append(out.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    y_pred_binary = (torch.cat(y_pred) > 0.5).numpy()
    y_true_array = torch.cat(y_true).numpy()

    auc = roc_auc_score(y_true_array, torch.cat(y_pred))
    accuracy = accuracy_score(y_true_array, y_pred_binary)
    f1 = f1_score(y_true_array, y_pred_binary)
    precision = precision_score(y_true_array, y_pred_binary)
    recall = recall_score(y_true_array, y_pred_binary)

    return auc, accuracy, f1, precision, recall
    

In [10]:
import matplotlib.pyplot as plt
train_loss = []
for epoch in range(300):
    loss = train()
    val_results = test(val_loader)
    val_auc, val_accuracy, val_f1, val_precision, val_recall = val_results
    print(f'Epoch {epoch:>2} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} ')
    train_loss.append(loss)

test_results = test(test_loader)
test_auc, test_accuracy, test_f1, test_precision, test_recall = test_results 
print(f'Test AUC: {test_auc:.4f} | Test Accuracy: {test_accuracy:.4f} | Test F1: {test_f1:.4f} | Test Precision: {test_precision:.4f} | Test Recall: {test_recall:.4f}')

Epoch  0 | Loss: 0.6046 | Val AUC: 0.7003 
Epoch  1 | Loss: 0.5472 | Val AUC: 0.6964 
Epoch  2 | Loss: 0.5444 | Val AUC: 0.6889 
Epoch  3 | Loss: 0.5432 | Val AUC: 0.6921 
Epoch  4 | Loss: 0.5422 | Val AUC: 0.6938 
Epoch  5 | Loss: 0.5418 | Val AUC: 0.6857 
Epoch  6 | Loss: 0.5418 | Val AUC: 0.6967 
Epoch  7 | Loss: 0.5410 | Val AUC: 0.6977 
Epoch  8 | Loss: 0.5412 | Val AUC: 0.6835 
Epoch  9 | Loss: 0.5411 | Val AUC: 0.6943 
Epoch 10 | Loss: 0.5408 | Val AUC: 0.6936 
Epoch 11 | Loss: 0.5411 | Val AUC: 0.6921 
Epoch 12 | Loss: 0.5404 | Val AUC: 0.6944 
Epoch 13 | Loss: 0.5407 | Val AUC: 0.6829 
Epoch 14 | Loss: 0.5403 | Val AUC: 0.6987 
Epoch 15 | Loss: 0.5402 | Val AUC: 0.6945 
Epoch 16 | Loss: 0.5404 | Val AUC: 0.6985 
Epoch 17 | Loss: 0.5402 | Val AUC: 0.6950 
Epoch 18 | Loss: 0.5401 | Val AUC: 0.6947 
Epoch 19 | Loss: 0.5398 | Val AUC: 0.6968 
Epoch 20 | Loss: 0.5398 | Val AUC: 0.6975 
Epoch 21 | Loss: 0.5394 | Val AUC: 0.7081 
Epoch 22 | Loss: 0.5400 | Val AUC: 0.7060 
Epoch 23 | 